In [8]:
import cv2
import dlib
import os
import sys
import random
# 存储位置
output_dir = 'D:/No1WorkSpace/JupyterNotebook/Facetrainset/4Giao'
#output_dir = 'D:/No1WorkSpace/JupyterNotebook/Facetrainset/2Chle'
#
size = 256
 
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# 改变图片的亮度与对比度
 
def relight(img, light=1, bias=0):
    w = img.shape[1]
    h = img.shape[0]
    #image = []
    for i in range(0,w):
        for j in range(0,h):
            for c in range(3):
                tmp = int(img[j,i,c]*light + bias)
                if tmp > 255:
                    tmp = 255
                elif tmp < 0:
                    tmp = 0
                img[j,i,c] = tmp
    return img
 
#使用dlib自带的frontal_face_detector作为我们的特征提取器
detector = dlib.get_frontal_face_detector()
# 打开摄像头 参数为输入流，可以为摄像头或视频文件
camera = cv2.VideoCapture(0)
#camera = cv2.VideoCapture('C:/Users/CUNGU/Videos/Captures/wang.mp4')

index = 1
while True:
    if (index <= 20):#存储15张人脸特征图像
        print('Being processed picture %s' % index)
        # 从摄像头读取照片
        success, img = camera.read()
        # 转为灰度图片
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 使用detector进行人脸检测
        dets = detector(gray_img, 1)
 
        for i, d in enumerate(dets):
            x1 = d.top() if d.top() > 0 else 0
            y1 = d.bottom() if d.bottom() > 0 else 0
            x2 = d.left() if d.left() > 0 else 0
            y2 = d.right() if d.right() > 0 else 0
 
            face = img[x1:y1,x2:y2]
            # 调整图片的对比度与亮度， 对比度与亮度值都取随机数，这样能增加样本的多样性
            face = relight(face, random.uniform(0.5, 1.5), random.randint(-50, 50))
 
            face = cv2.resize(face, (size,size))
 
            cv2.imshow('image', face)
 
            cv2.imwrite(output_dir+'/'+str(index)+'.jpg', face)
 
            index += 1
        key = cv2.waitKey(30) & 0xff
        if key == 27:
            break
    else:
        print('Finished!')
        # 释放摄像头 release camera
        camera.release()
        # 删除建立的窗口 delete all the windows
        cv2.destroyAllWindows()
        break


Being processed picture 1
Being processed picture 2
Being processed picture 3
Being processed picture 4
Being processed picture 5
Being processed picture 6
Being processed picture 7
Being processed picture 8
Being processed picture 9
Being processed picture 10
Being processed picture 11
Being processed picture 12
Being processed picture 13
Being processed picture 14
Being processed picture 15
Being processed picture 16
Being processed picture 17
Being processed picture 18
Being processed picture 19
Being processed picture 20
Finished!


In [9]:
# 从人脸图像文件中提取人脸特征存入 CSV
# Features extraction from images and save into features_all.csv

# return_128d_features()          获取某张图像的128D特征
# compute_the_mean()              计算128D特征均值

from cv2 import cv2 as cv2
import os
import dlib
from skimage import io
import csv
import numpy as np

# 要读取人脸图像文件的路径
path_images_from_camera = "D:/No1WorkSpace/JupyterNotebook/Facetrainset/"

# Dlib 正向人脸检测器
detector = dlib.get_frontal_face_detector()

# Dlib 人脸预测器
predictor = dlib.shape_predictor("D:/No1WorkSpace/JupyterNotebook/model/shape_predictor_68_face_landmarks.dat")

# Dlib 人脸识别模型
# Face recognition model, the object maps human faces into 128D vectors
face_rec = dlib.face_recognition_model_v1("D:/No1WorkSpace/JupyterNotebook/model/dlib_face_recognition_resnet_model_v1.dat")


# 返回单张图像的 128D 特征
def return_128d_features(path_img):
    img_rd = io.imread(path_img)
    img_gray = cv2.cvtColor(img_rd, cv2.COLOR_BGR2RGB)
    faces = detector(img_gray, 1)

    print("%-40s %-20s" % ("检测到人脸的图像 / image with faces detected:", path_img), '\n')

    # 因为有可能截下来的人脸再去检测，检测不出来人脸了
    # 所以要确保是 检测到人脸的人脸图像 拿去算特征
    if len(faces) != 0:
        shape = predictor(img_gray, faces[0])
        face_descriptor = face_rec.compute_face_descriptor(img_gray, shape)
    else:
        face_descriptor = 0
        print("no face")

    return face_descriptor


# 将文件夹中照片特征提取出来, 写入 CSV
def return_features_mean_personX(path_faces_personX):
    features_list_personX = []
    photos_list = os.listdir(path_faces_personX)
    if photos_list:
        for i in range(len(photos_list)):
            with open("D:/No1WorkSpace/JupyterNotebook/feature/featuresGiao"+str(i)+".csv", "w", newline="") as csvfile:
                writer = csv.writer(csvfile)
            # 调用return_128d_features()得到128d特征
                print("%-40s %-20s" % ("正在读的人脸图像 / image to read:", path_faces_personX + "/" + photos_list[i]))
                features_128d = return_128d_features(path_faces_personX + "/" + photos_list[i])
                print(features_128d)
                writer.writerow(features_128d)
            # 遇到没有检测出人脸的图片跳过
                if features_128d == 0:
                    i += 1
                else:
                    features_list_personX.append(features_128d)
    else:
        print("文件夹内图像文件为空 / Warning: No images in " + path_faces_personX + '/', '\n')

    # 计算 128D 特征的均值
    # N x 128D -> 1 x 128D
    if features_list_personX:
        features_mean_personX = np.array(features_list_personX).mean(axis=0)
    else:
        features_mean_personX = '0'

    return features_mean_personX


# 读取某人所有的人脸图像的数据
people = os.listdir(path_images_from_camera)
people.sort()

with open("D:/No1WorkSpace/JupyterNotebook/feature/features_all.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for person in people:
        print("##### " + person + " #####")
        # Get the mean/average features of face/personX, it will be a list with a length of 128D
        features_mean_personX = return_features_mean_personX(path_images_from_camera + person)
        writer.writerow(features_mean_personX)
        print("特征均值 / The mean of features:", list(features_mean_personX))
        print('\n')
    print("所有录入人脸数据存入 / Save all the features of faces registered into: D:/myworkspace/JupyterNotebook/People/feature/features_all2.csv")


##### 3Wang #####
正在读的人脸图像 / image to read:                D:/No1WorkSpace/JupyterNotebook/Facetrainset/3Wang/1.jpg
检测到人脸的图像 / image with faces detected:    D:/No1WorkSpace/JupyterNotebook/Facetrainset/3Wang/1.jpg 

-0.219508
0.104045
0.058318
-0.0498521
-0.162905
-0.0639812
-0.0633557
-0.141306
0.0905059
-0.110819
0.221706
-0.0652917
-0.187473
-0.00457235
-0.0864505
0.226993
-0.176736
-0.197752
-0.0573079
-0.0142642
0.0798245
0.0233934
0.00234589
0.0450477
-0.0268765
-0.311612
-0.0763849
-0.0569146
0.0841115
-0.0174647
-0.0359196
0.0196265
-0.203557
-0.0629805
0.0371422
0.0832686
0.0213829
-0.0494738
0.229214
-0.0672526
-0.257904
0.0388988
0.110752
0.274492
0.1856
0.0777536
0.00397457
-0.107528
0.0712619
-0.193404
0.00844125
0.129611
0.124769
0.0172379
-0.0183483
-0.185771
0.0640226
0.104258
-0.145288
0.00668597
0.102998
-0.10626
0.0310345
-0.077996
0.308407
-0.0031676
-0.114096
-0.115645
0.155122
-0.0814511
-0.0523986
0.033662
-0.174436
-0.16556
-0.317804
0.006375
0.438143
0.109592
-

-0.140561
0.0844505
0.0782394
-0.0555946
-0.0972027
-0.0574167
-0.051037
-0.121414
0.101574
-0.0857296
0.189023
-0.0745118
-0.178536
-0.00321932
-0.0906128
0.2189
-0.178438
-0.161182
-0.0673076
-0.0200425
0.102331
0.0405118
-0.0275167
0.0364231
-0.0252751
-0.30838
-0.0888221
-0.0345899
0.0874216
-0.000797726
-0.0583021
0.0256113
-0.203702
-0.0602703
0.0424704
0.117004
0.00403894
-0.0891623
0.158074
-0.0550827
-0.241518
0.0678381
0.11013
0.23465
0.204708
0.0713841
0.0310988
-0.106332
0.0558105
-0.167149
-0.00346431
0.139419
0.0966715
0.0939979
-0.0157504
-0.178877
0.0661176
0.0881782
-0.125411
0.019783
0.116702
-0.0992437
0.0558128
-0.0504044
0.282126
0.0209829
-0.125105
-0.153828
0.16637
-0.126083
-0.0814769
-8.96147e-05
-0.172387
-0.128123
-0.317764
0.0318798
0.424198
0.0896377
-0.169402
0.131353
-0.0253687
0.0500027
0.129307
0.160279
-0.0221316
0.012643
-0.114631
-0.00572388
0.243919
-0.04648
-0.04015
0.156488
0.00365543
0.0405344
0.0300159
0.0828292
-0.0335457
0.0213915
-0.123738
0.

-0.159943
0.0535067
0.0889323
-0.0601529
-0.10648
-0.0639797
-0.0481316
-0.142488
0.103939
-0.112619
0.221169
-0.0699439
-0.216718
-0.0578284
-0.0733277
0.224936
-0.138423
-0.16525
-0.0590114
-0.0094234
0.0808405
-0.0283421
-0.0111576
0.0457165
-0.0367692
-0.308713
-0.104913
-0.0448664
0.0546058
-0.0320095
-0.0855518
-0.00767407
-0.222159
-0.0746254
0.0126359
0.0764401
0.0386769
-0.102905
0.157771
-0.037655
-0.224331
0.0300052
0.058129
0.224132
0.1784
0.0609539
0.0517194
-0.103162
0.0855398
-0.191492
0.0177978
0.107906
0.0891771
0.0439684
-0.0354874
-0.174488
0.055745
0.116869
-0.11199
0.0375078
0.119122
-0.103493
-0.016487
-0.0723989
0.285919
0.0305516
-0.143306
-0.142078
0.134062
-0.0885541
-0.0446083
-0.0248274
-0.197164
-0.143684
-0.335371
0.064312
0.410364
0.116297
-0.199963
0.106907
-0.0853355
0.0940618
0.106125
0.152712
0.00974602
0.0300149
-0.122802
-0.00638917
0.202787
-0.0691897
-0.0132063
0.188961
-0.0126263
0.0563307
-0.0124568
0.0597965
-0.0576935
0.00280849
-0.0883243
0.0

-0.185217
0.111484
0.0762707
-0.0524374
-0.119261
-0.0779026
-0.0657202
-0.133815
0.105919
-0.0991536
0.20378
-0.0763936
-0.19843
-0.00639604
-0.0888946
0.216044
-0.177206
-0.172444
-0.0598501
-0.00699558
0.10387
0.0170199
-0.0237048
0.0320875
-0.012173
-0.303743
-0.0895008
-0.038256
0.0808074
-0.00191655
-0.0326789
0.0152571
-0.198456
-0.0723537
0.0248557
0.0920633
0.0129963
-0.0710868
0.196146
-0.0664262
-0.249539
0.05236
0.0925642
0.254042
0.197634
0.0697675
0.020341
-0.112624
0.0745595
-0.182696
-0.0197875
0.15031
0.117587
0.0473336
-0.0132581
-0.191179
0.0695355
0.0990407
-0.133502
-0.000301546
0.111087
-0.118708
0.02938
-0.0711674
0.294061
0.00144663
-0.128415
-0.135958
0.15889
-0.0851836
-0.0588499
0.0174564
-0.180575
-0.139279
-0.32706
0.0212846
0.432958
0.0801452
-0.169798
0.142003
-0.054322
0.0436404
0.0972723
0.152534
-0.0247895
-0.0152218
-0.136108
-0.0143183
0.203009
-0.0733553
-0.018537
0.147108
0.0155029
0.0477901
-0.00236753
0.0685782
-0.0684327
0.0048764
-0.114445
0.01

-0.131871
0.0187048
0.0681845
-0.0721907
-0.106749
-0.0696398
-0.0461873
-0.145483
0.112084
-0.132864
0.172203
-0.0702888
-0.220465
-0.0939498
0.00981946
0.211659
-0.158979
-0.159409
-0.0915049
-0.00724974
-0.00514764
-0.00512331
0.0732298
0.0317188
-0.079321
-0.380246
-0.0759294
-0.113514
-0.0214587
0.020881
-0.0394691
0.032259
-0.21878
-0.0514005
0.0356655
0.135182
-0.0158463
-0.0985564
0.192804
-0.0564573
-0.290282
-0.0725431
0.0768462
0.152214
0.174023
0.0419564
0.0422793
-0.127772
0.099742
-0.147709
0.0103975
0.0618749
-0.00340222
0.0206664
0.0230336
-0.141653
0.0682108
0.103384
-0.120455
-0.0783556
0.0781378
-0.137807
-0.0716328
-0.103946
0.274874
0.0690472
-0.105603
-0.15739
0.109733
-0.169549
-0.0652117
0.0570685
-0.120548
-0.101246
-0.367828
-0.0721231
0.359719
0.0485088
-0.168349
0.029816
-0.0374529
0.0188282
0.152666
0.184353
0.0118427
0.0637038
-0.0928895
-0.0457671
0.227847
-0.0809175
0.00795921
0.135557
-0.0312246
0.0517409
0.017362
0.029712
-0.0732486
0.00891939
-0.11126

-0.115596
0.0167771
0.0614745
-0.0860656
-0.120943
-0.0775974
-0.0230413
-0.163882
0.0896675
-0.154903
0.172649
-0.0771303
-0.246128
-0.0905257
-0.00139564
0.181732
-0.159585
-0.15484
-0.111951
-0.0260671
-0.0229266
0.00526682
0.0473044
0.021097
-0.0833084
-0.375419
-0.0810695
-0.10886
-0.0136309
0.0188419
-0.0306235
0.0389209
-0.19433
-0.0299084
0.0408992
0.137849
-0.054779
-0.0866647
0.196185
-0.0546468
-0.262122
-0.0837449
0.0526351
0.182645
0.180103
0.0247011
0.0450328
-0.12126
0.100859
-0.167782
0.0186463
0.0843029
-0.0136183
0.0314531
0.0376485
-0.103562
0.0569512
0.13249
-0.133006
-0.074227
0.0918931
-0.170862
-0.0868302
-0.0991679
0.25082
0.0793308
-0.106237
-0.139052
0.14129
-0.209871
-0.0899899
0.00970403
-0.105591
-0.0857486
-0.357643
-0.0716677
0.361322
0.0772685
-0.148239
0.0142138
-0.030886
0.00996596
0.179762
0.149465
-0.0130619
0.0367654
-0.087795
-0.00704835
0.231509
-0.0889446
0.0132957
0.1206
-0.0282649
0.0285708
0.0357962
0.00527277
-0.0714088
-0.00318081
-0.114081


-0.139351
0.0379357
0.0934521
-0.11297
-0.0982432
-0.102313
-0.0485977
-0.135956
0.121941
-0.144591
0.156551
-0.0739597
-0.187859
-0.0729536
2.45702e-05
0.175479
-0.119024
-0.148792
-0.0989096
0.00605448
0.00348874
0.00308014
0.0727286
0.0360854
-0.0692887
-0.373113
-0.0677097
-0.0792307
-0.0519975
0.0313394
-0.0255794
0.0698678
-0.207637
-0.0683649
0.00176331
0.142658
-0.0185452
-0.0923736
0.196188
-0.0530045
-0.301718
-0.0540367
0.0683881
0.207339
0.195994
-0.00148138
0.057413
-0.0996886
0.114116
-0.182113
0.0115679
0.0535405
0.00575753
0.00473192
0.0521781
-0.146163
0.108905
0.0877842
-0.101836
-0.066825
0.0928137
-0.131114
-0.0442597
-0.0903411
0.26308
0.0722036
-0.0793359
-0.166332
0.0882837
-0.161061
-0.057378
0.0714292
-0.154883
-0.0991794
-0.384353
-0.0609047
0.341292
0.0804382
-0.157773
0.0569717
-0.0435284
0.0440493
0.16537
0.177674
-0.00379473
0.0815587
-0.110014
-0.0217694
0.227562
-0.0654007
-0.040219
0.149105
0.00494379
0.0431722
0.0146586
0.000142604
-0.0658614
0.0284017

In [11]:
# 摄像头实时人脸识别
import os
import dlib          # 人脸处理的库 Dlib
import csv # 存入表格
import time
import sys
import numpy as np   # 数据处理的库 numpy
from cv2 import cv2 as cv2           # 图像处理的库 OpenCv
import pandas as pd  # 数据处理的库 Pandas


# 人脸识别模型，提取128D的特征矢量
# face recognition model, the object maps human faces into 128D vectors
# Refer this tutorial: http://dlib.net/python/index.html#dlib.face_recognition_model_v1
facerec = dlib.face_recognition_model_v1("D:/No1WorkSpace/JupyterNotebook/model/dlib_face_recognition_resnet_model_v1.dat")


# 计算两个128D向量间的欧式距离
# compute the e-distance between two 128D features
def return_euclidean_distance(feature_1, feature_2):
    feature_1 = np.array(feature_1)
    feature_2 = np.array(feature_2)
    dist = np.sqrt(np.sum(np.square(feature_1 - feature_2)))
    return dist


# 处理存放所有人脸特征的 csv
path_features_known_csv = "D:/No1WorkSpace/JupyterNotebook/feature/features_all.csv"
csv_rd = pd.read_csv(path_features_known_csv, header=None)


# 用来存放所有录入人脸特征的数组
# the array to save the features of faces in the database
features_known_arr = []

# 读取已知人脸数据
# print known faces
for i in range(csv_rd.shape[0]):
    features_someone_arr = []
    for j in range(0, len(csv_rd.loc[i, :])):
        features_someone_arr.append(csv_rd.loc[i, :][j])
    features_known_arr.append(features_someone_arr)
print("Faces in Database：", len(features_known_arr))

# Dlib 检测器和预测器
# The detector and predictor will be used
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('D:/No1WorkSpace/JupyterNotebook/model/shape_predictor_68_face_landmarks.dat')

# 创建 cv2 摄像头对象
# cv2.VideoCapture(0) to use the default camera of PC,
# and you can use local video name by use cv2.VideoCapture(filename)
cap = cv2.VideoCapture(0)

# cap.set(propId, value)
# 设置视频参数，propId 设置的视频参数，value 设置的参数值
cap.set(3, 480)

# cap.isOpened() 返回 true/false 检查初始化是否成功
# when the camera is open
while cap.isOpened():

    flag, img_rd = cap.read()
    kk = cv2.waitKey(1)

    # 取灰度
    img_gray = cv2.cvtColor(img_rd, cv2.COLOR_RGB2GRAY)

    # 人脸数 faces
    faces = detector(img_gray, 0)

    # 待会要写的字体 font to write later
    font = cv2.FONT_HERSHEY_COMPLEX

    # 存储当前摄像头中捕获到的所有人脸的坐标/名字
    # the list to save the positions and names of current faces captured
    pos_namelist = []
    name_namelist = []

    # 按下 q 键退出
    # press 'q' to exit
    if kk == ord('q'):
        break
    else:
        # 检测到人脸 when face detected
        if len(faces) != 0:  
            # 获取当前捕获到的图像的所有人脸的特征，存储到 features_cap_arr
            # get the features captured and save into features_cap_arr
            features_cap_arr = []
            for i in range(len(faces)):
                shape = predictor(img_rd, faces[i])
                features_cap_arr.append(facerec.compute_face_descriptor(img_rd, shape))

            # 遍历捕获到的图像中所有的人脸
            # traversal all the faces in the database
            for k in range(len(faces)):
                print("##### camera person", k+1, "#####")
                # 让人名跟随在矩形框的下方
                # 确定人名的位置坐标
                # 先默认所有人不认识，是 unknown
                # set the default names of faces with "unknown"
                name_namelist.append("unknown")

                # 每个捕获人脸的名字坐标 the positions of faces captured
                pos_namelist.append(tuple([faces[k].left(), int(faces[k].bottom() + (faces[k].bottom() - faces[k].top())/4)]))

                # 对于某张人脸，遍历所有存储的人脸特征
                # for every faces detected, compare the faces in the database
                e_distance_list = []
                for i in range(len(features_known_arr)):
                    # 如果 person_X 数据不为空
                    if str(features_known_arr[i][0]) != '0.0':
                        print("with person", str(i + 1), "the e distance: ", end='')
                        e_distance_tmp = return_euclidean_distance(features_cap_arr[k], features_known_arr[i])
                        print(e_distance_tmp)
                        e_distance_list.append(e_distance_tmp)
                    else:
                        # 空数据 person_X
                        e_distance_list.append(999999999)
                # 找出最接近的一个人脸数据是第几个
                # Find the one with minimum e distance
                similar_person_num = e_distance_list.index(min(e_distance_list))
                print("Minimum e distance with person", int(similar_person_num)+1)
                
                # 计算人脸识别特征与数据集特征的欧氏距离
                # 距离小于0.4则标出为可识别人物
                if min(e_distance_list) < 0.4:
                    # 这里可以修改摄像头中标出的人名
                    # Here you can modify the names shown on the camera
                    # 1、遍历文件夹目录
                    folder_name = 'D:/No1WorkSpace/JupyterNotebook/Facetrainset/'
                    # 最接近的人脸
                    sum=similar_person_num+1
                    key_id=1 # 从第一个人脸数据文件夹进行对比
                    # 获取文件夹中的文件名:1wang、2zhou、3...
                    file_names = os.listdir(folder_name)
                    for name in file_names:
                        # print(name+'->'+str(key_id))
                        if sum ==key_id:
                            #winsound.Beep(300,500)# 响铃：300频率，500持续时间
                            name_namelist[k] = name[1:]#人名删去第一个数字（用于视频输出标识）
                        key_id += 1
                    # 播放欢迎光临音效
                    #playsound('D:/myworkspace/JupyterNotebook/People/music/welcome.wav')
                    # print("May be person "+str(int(similar_person_num)+1))
                    # -----------筛选出人脸并保存到visitor文件夹------------
                    for i, d in enumerate(faces):
                        x1 = d.top() if d.top() > 0 else 0
                        y1 = d.bottom() if d.bottom() > 0 else 0
                        x2 = d.left() if d.left() > 0 else 0
                        y2 = d.right() if d.right() > 0 else 0
                        face = img_rd[x1:y1,x2:y2]
                        size = 64
                        face = cv2.resize(face, (size,size))
                        # 要存储visitor人脸图像文件的路径
                        path_visitors_save_dir = "D:/No1WorkSpace/JupyterNotebook/KnownFacetrainset/"
                        # 存储格式：2019-06-24-14-33-40wang.jpg
                        now_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
                        save_name = str(now_time)+str(name_namelist[k])+'.jpg'
                        # print(save_name)
                        # 本次图片保存的完整url
                        save_path = path_visitors_save_dir+'/'+ save_name    
                        # 遍历visitor文件夹所有文件名
                        visitor_names = os.listdir(path_visitors_save_dir)
                        visitor_name=''
                        for name in visitor_names:
                            # 名字切片到分钟数：2019-06-26-11-33-00wangyu.jpg
                            visitor_name=(name[0:16]+'-00'+name[19:])
                        # print(visitor_name)
                        visitor_save=(save_name[0:16]+'-00'+save_name[19:])
                        # print(visitor_save)
                        # 一分钟之内重复的人名不保存
                        if visitor_save!=visitor_name:
                            cv2.imwrite(save_path, face)
                            print('新存储：'+path_visitors_save_dir+'/'+str(now_time)+str(name_namelist[k])+'.jpg')
                        else:
                            print('重复，未保存！')
                            
                else:
                    # 播放无法识别音效
                    #playsound('D:/myworkspace/JupyterNotebook/People/music/sorry.wav')
                    print("Unknown person")
                    # -----保存图片-------
                    # -----------筛选出人脸并保存到visitor文件夹------------
                    for i, d in enumerate(faces):
                        x1 = d.top() if d.top() > 0 else 0
                        y1 = d.bottom() if d.bottom() > 0 else 0
                        x2 = d.left() if d.left() > 0 else 0
                        y2 = d.right() if d.right() > 0 else 0
                        face = img_rd[x1:y1,x2:y2]
                        size = 64
                        face = cv2.resize(face, (size,size))
                        # 要存储visitor-》unknown人脸图像文件的路径
                        path_visitors_save_dir = "D:/No1WorkSpace/JupyterNotebook/UnKnownFacetrainset/"
                        # 存储格式：2019-06-24-14-33-40unknown.jpg
                        now_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
                        # print(save_name)
                        # 本次图片保存的完整url
                        save_path = path_visitors_save_dir+'/'+ str(now_time)+'unknown.jpg'
                        cv2.imwrite(save_path, face)
                        print('新存储：'+path_visitors_save_dir+'/'+str(now_time)+'unknown.jpg')
                
                # 矩形框
                # draw rectangle
                for kk, d in enumerate(faces):
                    # 绘制矩形框
                    cv2.rectangle(img_rd, tuple([d.left(), d.top()]), tuple([d.right(), d.bottom()]), (0, 255, 255), 2)
                print('\n')

            # 在人脸框下面写人脸名字
            # write names under rectangle
            for i in range(len(faces)):
                cv2.putText(img_rd, name_namelist[i], pos_namelist[i], font, 0.8, (0, 255, 255), 1, cv2.LINE_AA)

    print("Faces in camera now:", name_namelist, "\n")

    #cv2.putText(img_rd, "Press 'q': Quit", (20, 450), font, 0.8, (84, 255, 159), 1, cv2.LINE_AA)
    cv2.putText(img_rd, "Face Recognition", (20, 40), font, 1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(img_rd, "Visitors: " + str(len(faces)), (20, 100), font, 1, (0, 0, 255), 1, cv2.LINE_AA)

    # 窗口显示 show with opencv
    cv2.imshow("camera", img_rd)

# 释放摄像头 release camera
cap.release()

# 删除建立的窗口 delete all the windows
cv2.destroyAllWindows()


Faces in Database： 2
##### camera person 1 #####
with person 1 the e distance: 0.3691990167291081
with person 2 the e distance: 0.5358717872131274
Minimum e distance with person 1
新存储：D:/No1WorkSpace/JupyterNotebook/KnownFacetrainset//2021-11-14-20-49-48Karmen.jpg


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.3329365916795957
with person 2 the e distance: 0.5412047872904567
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.35260312988268433
with person 2 the e distance: 0.5372564537819384
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.33922120733396177
with person 2 the e distance: 0.5091302203034112
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in c

##### camera person 1 #####
with person 1 the e distance: 0.3207685430696336
with person 2 the e distance: 0.5320516238832115
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.31363275675316676
with person 2 the e distance: 0.54110133107132
Minimum e distance with person 1
重复，未保存！
重复，未保存！


##### camera person 2 #####
with person 1 the e distance: 0.6656015052843326
with person 2 the e distance: 0.648307313131266
Minimum e distance with person 2
Unknown person
新存储：D:/No1WorkSpace/JupyterNotebook/UnKnownFacetrainset//2021-11-14-20-50-06unknown.jpg
新存储：D:/No1WorkSpace/JupyterNotebook/UnKnownFacetrainset//2021-11-14-20-50-06unknown.jpg


Faces in camera now: ['Karmen', 'unknown'] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.5155584114789744
with person 2 the e distance: 0.5559327801609092
Minimum e distance with person 1
Unknown person
新存储：D:/No1WorkSpace/Jupyt

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.33214034899536754
with person 2 the e distance: 0.5847534904586074
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.3715997877950744
with person 2 the e distance: 0.5824179708214601
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.3468909884649753
with person 2 the e distance: 0.5627027237588801
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.4245804186371794
with person 2 the e distance: 0.5971017905433933
Minimum e distance with person 1
Unknown person
新存储：D:/No1WorkSpace/JupyterNotebook/UnKnownFacetrainset//2021-11-14-20-50-20unknown.jpg


Faces in camera now: ['unknown'] 

##### camera 

##### camera person 1 #####
with person 1 the e distance: 0.31372609416115715
with person 2 the e distance: 0.4716695907656482
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.36006833331296123
with person 2 the e distance: 0.4771367544269164
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.3442330276724084
with person 2 the e distance: 0.47347703222754606
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

Faces in camera now: [] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.3326632827414349
with person 2 the e distance: 0.46941214283067795
Minimum e distance with person 1
重复，未保存！


Faces in camera now: ['Karmen'] 

##### camera person 1 #####
with person 1 the e distance: 0.33517694881730753
with person 2 the e distance: 0.4495199111235963
Minim